In [4]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import glob


In [10]:
def calculate_variables(data, data_type, path=None, filename=None):
    
    def compute_biomechanics(force, dt, m_body, bw):
        max_force = np.max(force, axis=1)
        a = (force - bw) / m_body
        v = sp.integrate.cumtrapz(a, dx=dt)  # Adding initial value for matching array size
        max_v = np.max(v, axis = 1)
        print(max_v)
        F_trimmed = force - bw
        peak_power = np.dot(max_force, max_v)
        concentric_impulse = sp.integrate.cumtrapz(force, dx=dt)  # Cumulative impulse
        E_RFD = np.max(np.diff(force) / dt, axis=1)  # Taking max along axis 1
        
        return max_force, peak_power, np.max(concentric_impulse, axis=1), E_RFD
   
    def player_exists_in_directory(player_name, directory_path):
        # List all files in the directory
        files_in_directory = os.listdir(directory_path)

        # Check if any filename starts with the player's name
        for file in files_in_directory:
            if file.startswith(player_name):
                return True  # A file with the player's name exists in the directory

        return False  # No file with the player's name was found in the directory


    dt = 0.005
    bw = data['Time_s_'][4]
    m_body = round(bw/9.81, 2)
    


    if data_type == 'hitting':
        if 'Right_handed' in path:
            drive_direction = 'R'
            stride_direction = 'L'
        else:
            drive_direction = 'L'
            stride_direction = 'R'
        
        # Ensure column existence before extracting data to prevent potential KeyErrors
        columns_to_check_drive = [f'RawFx_{drive_direction}_N_', f'RawFy_{drive_direction}_N_', f'RawFz_{drive_direction}_N_']
        columns_to_check_stride = [f'RawFx_{stride_direction}_N_', f'RawFy_{stride_direction}_N_', f'RawFz_{stride_direction}_N_']
        
        for col in columns_to_check_drive:
            if col not in data.columns:
                raise KeyError(f"Column '{col}' not found in the DataFrame for drive leg")
        for col in columns_to_check_stride:
            if col not in data.columns:
                raise KeyError(f"Column '{col}' not found in the DataFrame for stride leg")
            
        force_drive = np.array([data[f'RawFx_{drive_direction}_N_'][9:].values, 
                                data[f'RawFy_{drive_direction}_N_'][9:].values, 
                                data[f'RawFz_{drive_direction}_N_'][9:].values])
        
        force_stride = np.array([data[f'RawFx_{stride_direction}_N_'][9:].values, 
                                data[f'RawFy_{stride_direction}_N_'][9:].values, 
                                data[f'RawFz_{stride_direction}_N_'][9:].values])
        
        max_force_drive, peak_power_drive, max_impulse_drive, _ = compute_biomechanics(force_drive, dt, m_body, bw)
        max_force_stride, peak_power_stride, max_impulse_stride, _ = compute_biomechanics(force_stride, dt, m_body, bw)

        return pd.DataFrame({
            'mass': m_body,
            'hitting_drive_Fx': max_force_drive[0],
            'hitting_drive_Fy': max_force_drive[1],
            'hitting_drive_Fz': max_force_drive[2],
            'hitting_stride_Fx': max_force_stride[0],
            'hitting_stride_Fy': max_force_stride[1],
            'hitting_stride_Fz': max_force_stride[2],
            'hitting_peak_power_stride': peak_power_stride,
            'hitting_peak_power_drive': peak_power_drive,
            'hitting_max_impulse_x': max_impulse_stride[0],
            'hitting_max_impulse_y': max_impulse_stride[1],
            'hitting_max_impulse_z': max_impulse_stride[2] 
        }, index=[0])

    
    elif data_type == 'jump':
        #print(filename)
        player_name = filename.split('-')[0].strip()  # This will give "LaRocca, Jake"
        #print(f'Player name: {player_name}')

        # Paths to the right-handed and left-handed directories
        right_handed_directory = '/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /Hitting_BASEBALL/Right_handed'
        left_handed_directory = '/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /Hitting_BASEBALL/Left_handed'

        # Check if player's name exists in the right_handed or left_handed directories
        player_in_right_handed = player_exists_in_directory(player_name, right_handed_directory)
        player_in_left_handed = player_exists_in_directory(player_name, left_handed_directory)

        #print(f"Checking for {player_name} in: {right_handed_directory}")
        #print(f"Exists? {player_in_right_handed}")
        print(path)
        if 'drive_leg' in path:
            print(f'Processing path: {path}')
            
            if player_in_right_handed:
                leg = 'R'
            elif player_in_left_handed:
                leg = 'L'
            else:
                leg = None
            prefix = 'lmj_drive'
        else:
            if player_in_right_handed:
                leg = 'L'
            elif player_in_left_handed:
                leg = 'R'
            else:
                leg = None
            prefix = 'lmj_stride'

        print(f'Player name: {player_name} : {leg}') 


        #print(np.max(data[f"RawFx_{leg}_N_"]))
        force_leg = np.array([data[f'RawFx_{leg}_N_'][9:], data[f'RawFy_{leg}_N_'][9:], data[f'RawFz_{leg}_N_'][9:]])
        print(force_leg)
        max_force_leg, peak_power_leg, max_impulse_leg, E_RFD_leg = compute_biomechanics(force_leg, dt, m_body,bw)

        ''' print({
            f'{prefix}_Fx': max_force_leg[0],
            f'{prefix}_Fy': max_force_leg[1],
            f'{prefix}_Fz': max_force_leg[2],
            f'{prefix}_peak_force': np.sum(max_force_leg),
            f'{prefix}_peak_power': peak_power_leg,
            f'{prefix}_max_impulse_x': max_impulse_leg[0],
            f'{prefix}_max_impulse_y': max_impulse_leg[1],
            f'{prefix}_max_impulse_z': max_impulse_leg[2],
            f'{prefix}_E-RFD_x': E_RFD_leg[0],
            f'{prefix}_E-RFD_y': E_RFD_leg[1],
            f'{prefix}_E-RFD_z': E_RFD_leg[2]
        })'''
        return pd.DataFrame({
            f'{prefix}_Fx': max_force_leg[0],
            f'{prefix}_Fy': max_force_leg[1],
            f'{prefix}_Fz': max_force_leg[2],
            f'{prefix}_peak_force': np.sum(max_force_leg),
            f'{prefix}_peak_power': peak_power_leg,
            f'{prefix}_max_impulse_x': max_impulse_leg[0],
            f'{prefix}_max_impulse_y': max_impulse_leg[1],
            f'{prefix}_max_impulse_z': max_impulse_leg[2],
            f'{prefix}_E-RFD_x': E_RFD_leg[0],
            f'{prefix}_E-RFD_y': E_RFD_leg[1],
            f'{prefix}_E-RFD_z': E_RFD_leg[2]
        }, index=[0])

    

    elif data_type == 'broad_jump':
        
        # Assume both legs exert force during a broad jump, so we sum forces from both legs.
        force_R = np.array([data['RawFx_R_N_'][9:], data['RawFy_R_N_'][9:], data['RawFz_R_N_'][9:]])
        force_L = np.array([data['RawFx_L_N_'][9:], data['RawFy_L_N_'][9:], data['RawFz_L_N_'][9:]])
       
        max_L = np.max(np.sqrt(np.sum(force_L**2, axis=0)))
        max_R = np.max(np.sqrt(np.sum(force_R**2, axis=0)))
        dt = 0.005
        bw = data['Time_s_'][4]
        m_body = round(bw/9.81, 2)
        
        force_total = force_R + force_L
        max_force_total, peak_power_total, max_impulse_total, E_RFD_total = compute_biomechanics(force_total, dt, m_body, bw)
        

        return pd.DataFrame({
            'broad_jump_Fx': max_force_total[0],
            'broad_jump_Fy': max_force_total[1],
            'broad_jump_Fz': max_force_total[2],
            'right_leg_max_force': max_R,
            'right_leg_force_max_x': np.max(data['RawFx_R_N_'][9:]),
            'right_leg_force_max_y': np.max(data['RawFy_R_N_'][9:]),
            'right_leg_force_max_z': np.max(data['RawFz_R_N_'][9:]),
            'left_leg_max_force': max_L,
            'left_leg_force_max_x': np.max(data['RawFx_L_N_'][9:]),
            'left_leg_force_max_y': np.max(data['RawFy_L_N_'][9:]),
            'left_leg_force_max_z': np.max(data['RawFz_L_N_'][9:]),
            'broad_jump_peak_force': np.sum(max_force_total),
            'broad_jump_peak_power': peak_power_total,
            'broad_jump_max_impulse_x': max_impulse_total[0],
            'broad_jump_max_impulse_y': max_impulse_total[1],
            'broad_jump_max_impulse_z': max_impulse_total[2],
            'broad_jump_E-RFD_x': E_RFD_total[0],
            'broad_jump_E-RFD_y': E_RFD_total[1],
            'broad_jump_E-RFD_z': E_RFD_total[2]
        }, index=[0])


In [11]:
def extract_name_from_filename(filename):
    # Extract the "Lastname, Firstname" part from the filename
    name_part = filename.split('-')[0].strip()
    
    # Split the name into last and first names
    last_name, first_name = name_part.split(',')
    #print(first_name.strip() + " " + last_name.strip())
    
    # Return formatted name "Firstname Lastname"
    return first_name.strip() + " " + last_name.strip()


def process_excel_files(path, jump_type):
    output_data = []
    
    for filename in os.listdir(path):
        if filename.endswith('.xlsx'):
            #print(f"Processing file: {filename}")  # to see which file you're processing
            
            player_name = extract_name_from_filename(filename)
            
            data = pd.read_excel(os.path.join(path, filename), engine='openpyxl')
            results = calculate_variables(data, jump_type, path, filename)
            
            # Add the player's name to the beginning of the results DataFrame
            results.insert(0, 'Player Name', player_name)
            
            output_data.append(results)

    return pd.concat(output_data, axis=0)


# ... [All the previous functions remain unchanged]

def main():
    right_handed_path = '/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /Hitting_BASEBALL/Right_handed'
    left_handed_path = '/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /Hitting_BASEBALL/Left_handed'
    
    lmj_drive_path = '/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /LMJ_BASEBALL/drive_leg'
    lmj_stride_path = '/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /LMJ_BASEBALL/stride_leg'
    broad_jump_path = '/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /BROAD_BASEBALL/hitters'
    
    right_handed_data = process_excel_files(right_handed_path, 'hitting')
    left_handed_data = process_excel_files(left_handed_path, 'hitting')
    lmj_drive_data = process_excel_files(lmj_drive_path, 'jump')
    lmj_stride_data = process_excel_files(lmj_stride_path, 'jump')
    broad_jump_data = process_excel_files(broad_jump_path, 'broad_jump')

    combined_data = pd.concat([right_handed_data, left_handed_data, lmj_drive_data, lmj_stride_data, broad_jump_data], axis=0)
    combined_data.reset_index(drop=True, inplace=True)
    
    # Group by player name and calculate the average for each metric
    averaged_data = combined_data.groupby('Player Name').mean().reset_index()

    # Save averaged_data to a new Excel file
    output_file_path = "/Users/charlesarnold/Library/Mobile Documents/com~apple~CloudDocs/Documents/Shared Documents/LA Tech/2023/Force_plate_Testing/BASEBALL /baseball_output_data.xlsx"
    averaged_data.to_excel(output_file_path, index=False)
    print(f"Data saved to {output_file_path}")

if __name__ == "__main__":
    main()



[-0.04172777 -0.04922018 -0.01919365]
[-0.05549096 -0.050001   -0.02777978]
[-0.04052891 -0.05067304 -0.02360808]
[-0.0561066  -0.0491034  -0.02505182]
[-0.04238354 -0.04797104 -0.02340802]
[-0.05555    -0.05064568 -0.02565113]
[-0.0440832  -0.04931274 -0.02644769]
[-0.0544016  -0.04891639 -0.02874435]
[-0.03918688 -0.05036313 -0.01302938]
[-0.05046352 -0.04849916 -0.03999445]
[-0.0490528  -0.04856869 -0.04904842]
[-0.06385405 -0.04736413 -0.00740611]
[-0.04373561 -0.04884833 -0.02301903]
[-0.05544364 -0.04922505 -0.02664751]
[-0.03600953 -0.0467046  -0.00894561]
[-0.04882598 -0.04895265 -0.04933981]
[-0.04429957 -0.04939263 -0.02580153]
[-0.05536984 -0.04924756 -0.02395751]
[-0.04186427 -0.05004528 -0.0222832 ]
[-0.05423289 -0.04835828 -0.02612181]
[-0.04343146 -0.04981437 -0.02922592]
[-0.06001001 -0.04901711 -0.02072717]
[-0.03722741 -0.04901143 -0.01018163]
[-0.05302203 -0.05054552 -0.03279003]
[-0.03982985 -0.04997583 -0.02157798]
[-0.05747758 -0.04836585 -0.02633879]
[-0.03905358